<a href="https://colab.research.google.com/github/ShabnaIlmi/Data-Science-Group-Project/blob/recipe-risk-analyzer/DSGP_startover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Load and Inspect the Data**

In [1]:
import pandas as pd

# Load datasets
recipes_path = "/content/recipes_nodup.csv"
chem_path = "/content/chem.csv"

df_recipes = pd.read_csv(recipes_path)
df_chem = pd.read_csv(chem_path)

# Display first few rows of each dataset
print("📌 recipes_nodup.csv:")
print(df_recipes.head(), "\n\n")

print("📌 chem.csv:")
print(df_chem.head(), "\n\n")

# Check dataset shapes
print(f"🔍 recipes_nodup.csv Shape: {df_recipes.shape}")
print(f"🔍 chem.csv Shape: {df_chem.shape}")

# Show column names
print("🛠 recipes_nodup Columns:", df_recipes.columns)
print("🛠 chem Columns:", df_chem.columns)

# Check missing values
print("⚠️ Missing values in recipes_nodup:\n", df_recipes.isnull().sum(), "\n")
print("⚠️ Missing values in chem:\n", df_chem.isnull().sum(), "\n")


📌 recipes_nodup.csv:
   Recipe ID                                    Chemical Names  \
0          1               Ephedrine + Red Phosphorus + Iodine   
1          2             Toluene + Nitric Acid + Sulfuric Acid   
2          3       Hydrogen Peroxide + Acetone + Sulfuric Acid   
3          4  Ephedrine + Potassium Permanganate + Acetic Acid   
4          5             Potassium Nitrate + Charcoal + Sulfur   

                     Formulas   Quantities (g/mL)  \
0           C10H15NO + P + I2     30g + 15g + 10g   
1         C7H8 + HNO3 + H2SO4  50mL + 30mL + 40mL   
2        H2O2 + C3H6O + H2SO4   20mL + 30mL + 5mL   
3  C10H15NO + KMnO4 + CH3COOH    25g + 10g + 50mL   
4                KNO3 + C + S     75g + 15g + 10g   

                         CAS Numbers    Solvent Used  \
0   299-42-3 + 7723-14-0 + 7553-56-2  Acetone, Ether   
1   108-88-3 + 7697-37-2 + 7664-93-9             NaN   
2    7722-84-1 + 67-64-1 + 7664-93-9             NaN   
3     299-42-3 + 7722-64-7 + 64-19-7   

In [2]:
# Check data types of columns
print("🔍 Data types in recipes_nodup.csv:\n", df_recipes.dtypes, "\n")
print("🔍 Data types in chem.csv:\n", df_chem.dtypes, "\n")


🔍 Data types in recipes_nodup.csv:
 Recipe ID                                       int64
Chemical Names                                 object
Formulas                                       object
Quantities (g/mL)                              object
CAS Numbers                                    object
Solvent Used                                   object
Reaction Conditions                            object
Toxicity Level                                 object
Flammability (Yes/No)                          object
Reactivity (Stable/Unstable)                   object
Explosiveness (1-10)                            int64
Health Risk Score (0-100)                       int64
Environmental Hazard (Yes/No)                  object
Dual Use Potential (Yes/No)                    object
Intended Use                                   object
Export Restriction (Yes/No)                    object
Controlled Substance (Yes/No)                  object
Risk Assessment Score (0-100)                 

**Check Missing Values and Remove Duplicates**

In [3]:
print("📌 Duplicate rows in recipes_nodup:", df_recipes.duplicated().sum())
print("📌 Duplicate rows in chem:", df_chem.duplicated().sum())

print("⚠️ Missing values in recipes_nodup:\n", df_recipes.isnull().sum(), "\n")
print("⚠️ Missing values in chem:\n", df_chem.isnull().sum(), "\n")



📌 Duplicate rows in recipes_nodup: 0
📌 Duplicate rows in chem: 0
⚠️ Missing values in recipes_nodup:
 Recipe ID                                       0
Chemical Names                                  0
Formulas                                        0
Quantities (g/mL)                               0
CAS Numbers                                     0
Solvent Used                                   89
Reaction Conditions                             0
Toxicity Level                                  0
Flammability (Yes/No)                           0
Reactivity (Stable/Unstable)                    0
Explosiveness (1-10)                            0
Health Risk Score (0-100)                       0
Environmental Hazard (Yes/No)                   0
Dual Use Potential (Yes/No)                     0
Intended Use                                    0
Export Restriction (Yes/No)                     0
Controlled Substance (Yes/No)                   0
Risk Assessment Score (0-100)                   

In [4]:
# Display unique values for categorical columns
print("Unique values in key columns (recipes_nodup):\n")
for col in df_recipes.columns:
    print(f"{col}: {df_recipes[col].nunique()} unique values")

print("\n🛠 Unique values in key columns (chem):\n")
for col in df_chem.columns:
    print(f"{col}: {df_chem[col].nunique()} unique values")


Unique values in key columns (recipes_nodup):

Recipe ID: 150 unique values
Chemical Names: 149 unique values
Formulas: 140 unique values
Quantities (g/mL): 85 unique values
CAS Numbers: 142 unique values
Solvent Used: 4 unique values
Reaction Conditions: 65 unique values
Toxicity Level: 5 unique values
Flammability (Yes/No): 2 unique values
Reactivity (Stable/Unstable): 3 unique values
Explosiveness (1-10): 10 unique values
Health Risk Score (0-100): 18 unique values
Environmental Hazard (Yes/No): 2 unique values
Dual Use Potential (Yes/No): 2 unique values
Intended Use: 106 unique values
Export Restriction (Yes/No): 2 unique values
Controlled Substance (Yes/No): 2 unique values
Risk Assessment Score (0-100): 18 unique values
Regulatory Body: 7 unique values
Compliance Status (Compliant/Non-compliant): 2 unique values
Risk Category: 4 unique values
Risk Score (0-100): 16 unique values

🛠 Unique values in key columns (chem):

ID: 401 unique values
Chemical name: 393 unique values
molar

**Preprocessing recipes_nodup.csv**

In [5]:
# Fill missing values with "Unknown"
df_recipes["Solvent Used"].fillna("Unknown", inplace=True)


<ipython-input-5-d9d1a59aa268>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_recipes["Solvent Used"].fillna("Unknown", inplace=True)


In [33]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# ✅ Load Dataset
recipes_df = df_recipes

# ✅ Step 1: Combine Chemical Names and Quantities
def combine_chemicals(row):
    chemicals = row['Chemical Names'].split(' + ')
    quantities = row['Quantities (g/mL)'].split(' + ')
    combined = [f"{chem.strip()}:{re.sub(r'[^0-9.]+', '', qty)}" for chem, qty in zip(chemicals, quantities)]
    return ' + '.join(combined)

recipes_df['Combined Recipe'] = recipes_df.apply(combine_chemicals, axis=1)

# ✅ Step 2: Define Features and Targets
X = recipes_df[['Combined Recipe']]
y = recipes_df[[
    "Flammability (Yes/No)", "Reactivity (Stable/Unstable)",
    "Explosiveness (1-10)", "Health Risk Score (0-100)",
    "Risk Score (0-100)", "Risk Category"
]]

# ✅ Step 3: Encode Categorical Targets (Yes/No, Stable/Unstable)
binary_cols = ["Flammability (Yes/No)", "Reactivity (Stable/Unstable)"]
for col in binary_cols:
    y.loc[:, col] = y[col].apply(lambda x: 1 if x in ["Yes", "Stable"] else 0)

# ✅ Step 4: Normalize Numerical Targets Safely
num_cols = ["Explosiveness (1-10)", "Health Risk Score (0-100)", "Risk Score (0-100)"]
scaler = MinMaxScaler()
y.loc[:, num_cols] = scaler.fit_transform(y[num_cols])

# ✅ Step 5: Encode Risk Category (Low/Mid/High)
y.loc[:, 'Risk Category'] = y['Risk Category'].map({'Low': 0, 'Mid': 1, 'High': 2})

# ✅ Step 6: Split the Dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("✅ Dataset Prepared for Multi-Output Training.")


✅ Dataset Prepared for Multi-Output Training.


<ipython-input-33-edb432b873c1>:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.5 1.  1.  0.3 0.6 0.9 0.8 0.5 1.  0.8 1.  0.8 0.9 0.7 0.4 0.9 1.  0.6
 1.  0.7 1.  0.8 1.  0.8 0.9 0.5 0.9 0.6 0.9 0.5 0.5 0.8 0.7 1.  0.9 0.8
 0.9 0.9 1.  0.7 1.  0.9 0.8 0.7 1.  0.5 0.9 0.7 0.4 0.8 1.  1.  1.  1.
 0.9 0.9 0.5 1.  1.  1.  0.7 0.8 0.9 1.  0.9 1.  0.7 1.  1.  0.9 0.8 0.7
 1.  0.8 0.8 0.9 0.8 0.9 0.4 0.9 1.  1.  0.8 0.9 1.  1.  0.  0.  0.4 0.
 0.  0.  0.  0.  0.  0.  0.  0.3 0.  0.4 0.9 0.5 0.4 0.9 0.  0.  0.  0.
 0.2 0.3 0.5 0.7 0.7 0.9 0.  0.  0.  0.3 0.4 0.5 0.7 0.9 0.  0.  0.  0.
 0.3 0.5 0.7 0.8 0.  0.  0.  0.3 0.4 0.7 0.5 0.7 0.5 0.9 0.5 0.4 0.7 0.8
 1.  0.  0.  0.  0.  0.5]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  y.loc[:, num_cols] = scaler.fit_transform(y[num_cols])
<ipython-input-33-edb432b873c1>:35: FutureWarning: Setting an item of incompatible dtype is dep

In [ ]:
# Check for missing values in y_train
print("🔍 Missing Values in y_train:")
print(y_train.isnull().sum())


In [ ]:
# Fill missing numerical values with median and categorical with mode
y_train.fillna({
    "Explosiveness (1-10)": y_train["Explosiveness (1-10)"].median(),
    "Health Risk Score (0-100)": y_train["Health Risk Score (0-100)"].median(),
    "Risk Score (0-100)": y_train["Risk Score (0-100)"].median(),
    "Flammability (Yes/No)": y_train["Flammability (Yes/No)"].mode()[0],
    "Reactivity (Stable/Unstable)": y_train["Reactivity (Stable/Unstable)"].mode()[0],
    "Risk Category": y_train["Risk Category"].mode()[0]
}, inplace=True)


In [ ]:
# Split classification and regression targets
classification_targets = ["Flammability (Yes/No)", "Reactivity (Stable/Unstable)", "Risk Category"]
regression_targets = ["Explosiveness (1-10)", "Health Risk Score (0-100)", "Risk Score (0-100)"]

y_train_class = y_train[classification_targets]
y_train_reg = y_train[regression_targets]


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize and fit the vectorizer
vectorizer = TfidfVectorizer(max_features=100)
X_train_recipe_tfidf = vectorizer.fit_transform(X_train["Combined Recipe"]).toarray()

# Convert to DataFrame and reset the index
X_train_recipe_tfidf = pd.DataFrame(X_train_recipe_tfidf, columns=vectorizer.get_feature_names_out())
X_train_recipe_tfidf.reset_index(drop=True, inplace=True)

# Drop the original text column and combine with the TF-IDF features
X_train_processed = pd.concat([X_train.drop(columns=["Combined Recipe"]).reset_index(drop=True), X_train_recipe_tfidf], axis=1)

print("✅ TF-IDF Vectorization Done!")


In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
import joblib

# Train classification model
classifier = MultiOutputClassifier(RandomForestClassifier(random_state=42))
classifier.fit(X_train_processed, y_train_class)

# Save the classifier model
joblib.dump(classifier, 'multi_output_classifier_model.pkl')
print("✅ Classification Model Trained and Saved!")


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize and fit the vectorizer
vectorizer = TfidfVectorizer(max_features=100)
X_train_recipe_tfidf = vectorizer.fit_transform(X_train["Combined Recipe"]).toarray()

# Convert TF-IDF results to DataFrame
X_train_recipe_tfidf = pd.DataFrame(X_train_recipe_tfidf, columns=vectorizer.get_feature_names_out())
X_train_recipe_tfidf.reset_index(drop=True, inplace=True)

# Combine TF-IDF with other numerical features
X_train_final = pd.concat([X_train.drop(columns=["Combined Recipe"]).reset_index(drop=True), X_train_recipe_tfidf], axis=1)

print("✅ TF-IDF Vectorization Done for Regression!")


In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
import joblib

# Initialize and train the regression model
regressor = MultiOutputRegressor(RandomForestRegressor(random_state=42))
regressor.fit(X_train_final, y_train_reg)

# Save the trained model
joblib.dump(regressor, 'multi_output_regressor_model.pkl')
print("✅ Regression Model Trained and Saved!")


In [ ]:
import joblib

# Save the trained vectorizer
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')
print("✅ TF-IDF Vectorizer Saved!")


In [ ]:
def preprocess_new_recipe(new_recipe_raw):
    import re

    # Combine chemicals with quantities into the same format
    def combine_chemicals(chemicals, quantities):
        combined = [f"{chem.strip()}:{re.sub(r'[^0-9.]+', '', qty)}"
                    for chem, qty in zip(chemicals.split("+"), quantities.split("+"))]
        return " + ".join(combined)

    # Apply combination
    new_recipe_raw["Combined Recipe"] = new_recipe_raw.apply(
        lambda row: combine_chemicals(row["Chemical Names"], row["Quantities (g/mL)"]), axis=1
    )

    return new_recipe_raw

# Example new input
new_recipe_raw = pd.DataFrame({
    "Chemical Names": ["Copper Sulfate + Iron Nail"],
    "Quantities (g/mL)": ["300000000000 + 1000000000000"]
})

# Preprocess the input
new_recipe_processed = preprocess_new_recipe(new_recipe_raw)
print("✅ New Recipe Preprocessed!")


In [ ]:
# Load the saved vectorizer and model
vectorizer = joblib.load('tfidf_vectorizer.pkl')
regressor = joblib.load('multi_output_regressor_model.pkl')

# Vectorize the new recipe
new_recipe_tfidf = vectorizer.transform(new_recipe_processed["Combined Recipe"]).toarray()
new_recipe_tfidf = pd.DataFrame(new_recipe_tfidf, columns=vectorizer.get_feature_names_out())

# If there are additional numerical features, add them here
# (In this case, we only have TF-IDF features)
X_new = new_recipe_tfidf

print("✅ New Recipe Vectorized and Ready for Prediction!")


In [ ]:
# Perform the prediction
predictions = regressor.predict(X_new)

# Format and display the results
predicted_df = pd.DataFrame(predictions, columns=[
    "Explosiveness (1-10)", "Health Risk Score (0-100)", "Risk Score (0-100)"
])

print("\n🔍 Predicted Detailed Risk Analysis:")
print(predicted_df)


In [ ]:
def classify_risk(score):
    if score < 30:
        return "Low"
    elif 30 <= score < 70:
        return "Medium"
    else:
        return "High"

predicted_df["Risk Category"] = predicted_df["Risk Score (0-100)"].apply(classify_risk)
print("\n🔍 Final Prediction with Risk Category:")
print(predicted_df)


Encode Categorical Features

In [7]:
from sklearn.preprocessing import LabelEncoder

binary_cols = [
    "Flammability (Yes/No)", "Reactivity (Stable/Unstable)",
    "Environmental Hazard (Yes/No)", "Dual Use Potential (Yes/No)",
    "Export Restriction (Yes/No)", "Controlled Substance (Yes/No)",
    "Compliance Status (Compliant/Non-compliant)"
]

# Convert Yes/No to 0/1
for col in binary_cols:
    df_recipes[col] = df_recipes[col].apply(lambda x: 1 if x in ["Yes", "Compliant", "Stable"] else 0)

# Encode Risk Category
df_recipes["Risk Category Encoded"] = LabelEncoder().fit_transform(df_recipes["Risk Category"])


Normalize Numerical Features

In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
# Remove "Total Quantity (g/mL)" from num_cols as it is no longer present in df_recipes
num_cols = ["Risk Score (0-100)", "Health Risk Score (0-100)", "Risk Assessment Score (0-100)"]

df_recipes[num_cols] = scaler.fit_transform(df_recipes[num_cols])


# Save processed data
df_recipes.to_csv("/content/processed_recipes.csv", index=False)
print("✅ Successfully converted recipes into structured format!")

✅ Successfully converted recipes into structured format!


**Preprocessing chem.csv**

In [9]:
# Fill missing values in chem.csv
df_chem["CAS number"].fillna("Unknown", inplace=True)
df_chem["UN number"].fillna("Unknown", inplace=True)
df_chem["synonyms"].fillna("Unknown", inplace=True)

# Standardize column names
df_chem.columns = df_chem.columns.str.strip().str.lower().str.replace(" ", "_")

# Save preprocessed chem.csv
df_chem.to_csv("/content/preprocessed_chem.csv", index=False)
print("Preprocessed chem.csv saved!")


Preprocessed chem.csv saved!


<ipython-input-9-e602f27998e5>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_chem["CAS number"].fillna("Unknown", inplace=True)
<ipython-input-9-e602f27998e5>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

***Convert Recipes into Features***

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert "Combined Recipe" into TF-IDF vectors
vectorizer = TfidfVectorizer(max_features=500)  # Keep top 500 most important features
X_tfidf = vectorizer.fit_transform(df_recipes["Combined Recipe"]).toarray()

# Convert TF-IDF output to DataFrame and merge with other numerical features
X_tfidf_df = pd.DataFrame(X_tfidf, columns=vectorizer.get_feature_names_out())

# Drop "Combined Recipe" and merge TF-IDF features
df_final = pd.concat([df_recipes.drop(columns=["Combined Recipe"]), X_tfidf_df], axis=1)

print("✅ TF-IDF transformation complete!")


✅ TF-IDF transformation complete!


In [11]:
from sklearn.preprocessing import LabelEncoder
import joblib

# Define Label Encoder
label_encoder = LabelEncoder()

# Fit on the target variable (Risk Category)
df_recipes["Risk Category Encoded"] = label_encoder.fit_transform(df_recipes["Risk Category"])

# Save the trained LabelEncoder
joblib.dump(label_encoder, "risk_category_encoder.pkl")

print("✅ Label Encoder saved successfully!")


✅ Label Encoder saved successfully!


**Train ML Models**

In [12]:
import joblib

joblib.dump(vectorizer, "tfidf_vectorizer.pkl")
joblib.dump(scaler, "minmax_scaler.pkl")
joblib.dump(label_encoder, "risk_category_encoder.pkl")

print("✅ Preprocessing models saved!")


✅ Preprocessing models saved!


In [13]:
import re
import pandas as pd
import joblib
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the trained vectorizer, scaler, and label encoders
vectorizer = joblib.load("tfidf_vectorizer.pkl")
scaler = joblib.load("minmax_scaler.pkl")
label_encoder = joblib.load("risk_category_encoder.pkl")

def preprocess_new_input(df):
    df = df.copy()

    # Convert "Combined Recipe" into structured format
    if "Combined Recipe" in df.columns:
        def combine_chemicals(row):
            chemicals = row.split(" + ")
            combined = [f"{chem.split(':')[0]}:{re.sub(r'[^0-9.]', '', chem.split(':')[1])}" for chem in chemicals]
            return " + ".join(combined)

        df["Combined Recipe"] = df["Combined Recipe"].apply(combine_chemicals)

    # Convert categorical Yes/No features to 0/1
    binary_cols = [
        "Flammability (Yes/No)", "Reactivity (Stable/Unstable)",
        "Environmental Hazard (Yes/No)", "Dual Use Potential (Yes/No)",
        "Export Restriction (Yes/No)", "Controlled Substance (Yes/No)",
        "Compliance Status (Compliant/Non-compliant)"
    ]
    for col in binary_cols:
        if col in df.columns:
            df[col] = df[col].apply(lambda x: 1 if x in ["Yes", "Compliant", "Stable"] else 0)

    # Scale numerical columns
    num_cols = ["Risk Score (0-100)", "Health Risk Score (0-100)", "Risk Assessment Score (0-100)"]
    if set(num_cols).issubset(df.columns):
        df[num_cols] = scaler.transform(df[num_cols])  # Apply trained scaler

    # Transform text using trained TF-IDF vectorizer
    '''if "Combined Recipe" in df.columns:
        tfidf_features = vectorizer.transform(df["Combined Recipe"]).toarray()
        tfidf_df = pd.DataFrame(tfidf_features, columns=vectorizer.get_feature_names_out())
        df = pd.concat([df.drop(columns=["Combined Recipe"], errors="ignore"), tfidf_df], axis=1)

    return df'''


In [14]:
from sklearn.model_selection import train_test_split

# Define features (X) and target variable (y)
X = df_final.drop(columns=["Risk Category", "Risk Category Encoded"])  # Features
y = df_final["Risk Category Encoded"]  # Target

# Split into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("✅ Data split complete! Ready for training.")


✅ Data split complete! Ready for training.


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=100)

# Transform text columns
X_train_tfidf = vectorizer.fit_transform(X_train["Formulas"].fillna("")).toarray()
X_test_tfidf = vectorizer.transform(X_test["Formulas"].fillna("")).toarray()

# Convert to DataFrame and merge
X_train_tfidf_df = pd.DataFrame(X_train_tfidf, columns=vectorizer.get_feature_names_out())
X_test_tfidf_df = pd.DataFrame(X_test_tfidf, columns=vectorizer.get_feature_names_out())

X_train = pd.concat([X_train.drop(columns=["Formulas"]), X_train_tfidf_df], axis=1)
X_test = pd.concat([X_test.drop(columns=["Formulas"]), X_test_tfidf_df], axis=1)

print("✅ TF-IDF applied to text columns!")


✅ TF-IDF applied to text columns!


In [16]:
X_train = X_train.drop(columns=["Formulas", "CAS Numbers"], errors="ignore")
X_test = X_test.drop(columns=["Formulas", "CAS Numbers"], errors="ignore")

print("✅ Dropped Formulas & CAS Numbers (Not needed for model training).")


✅ Dropped Formulas & CAS Numbers (Not needed for model training).


In [17]:
# Combine features and target into one DataFrame if not done already
X = df_final.drop(columns=["Risk Category", "Risk Category Encoded"])
y = df_final["Risk Category Encoded"]

print("Before split - X shape:", X.shape, "y shape:", y.shape)

# Perform a stratified split to maintain class balance
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("After split - X_train shape:", X_train.shape, "y_train shape:", y_train.shape)


Before split - X shape: (150, 171) y shape: (150,)
After split - X_train shape: (120, 171) y_train shape: (120,)


In [18]:
from sklearn.preprocessing import LabelEncoder

# List of categorical columns
cat_cols = ["Solvent Used", "Reaction Conditions", "Toxicity Level", "Intended Use", "Regulatory Body"]

# Apply Label Encoding correctly
for col in cat_cols:
    encoder = LabelEncoder()

    # Fit on both training & testing data
    encoder.fit(pd.concat([X_train[col], X_test[col]], axis=0).astype(str))

    # Transform both datasets
    X_train[col] = encoder.transform(X_train[col].astype(str))
    X_test[col] = encoder.transform(X_test[col].astype(str))

print("✅ Successfully encoded categorical columns!")



✅ Successfully encoded categorical columns!


In [19]:
print("🔍 Data Types in X_train:\n", X_train.dtypes[X_train.dtypes == "object"])


🔍 Data Types in X_train:
 Formulas       object
CAS Numbers    object
dtype: object


In [20]:
drop_cols = ["Formulas", "CAS Numbers"]  # Add any other text-based columns

X_train = X_train.drop(columns=drop_cols, errors="ignore")
X_test = X_test.drop(columns=drop_cols, errors="ignore")

print("✅ Removed text columns from training data.")


✅ Removed text columns from training data.


In [21]:
X_train = X_train.astype(float)
X_test = X_test.astype(float)

print("✅ Confirmed all features are numeric!")


✅ Confirmed all features are numeric!


In [22]:
from imblearn.over_sampling import SMOTE

# Adjust SMOTE settings
smote = SMOTE(random_state=42, k_neighbors=1)  # Reduce k_neighbors
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Check new class distribution
import numpy as np
print("📊 Class distribution before SMOTE:", dict(zip(*np.unique(y_train, return_counts=True))))
print("📊 Class distribution after SMOTE:", dict(zip(*np.unique(y_train_balanced, return_counts=True))))



📊 Class distribution before SMOTE: {0: 65, 1: 13, 2: 23, 3: 19}
📊 Class distribution after SMOTE: {0: 65, 1: 65, 2: 65, 3: 65}


In [26]:
#RANDOM FORREST OPTIMIZED
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Train Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)



from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

# Grid search
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring="accuracy", n_jobs=-1)
grid_search.fit(X_train_balanced, y_train_balanced)

print("✅ Best Parameters:", grid_search.best_params_)

# Train optimized model
best_rf = RandomForestClassifier(**grid_search.best_params_, random_state=42)
best_rf.fit(X_train_balanced, y_train_balanced)

# Make predictions
y_pred_best_rf = best_rf.predict(X_test)

# Evaluate performance
print("📊 Optimized Random Forest Classification Report:\n", classification_report(y_test, y_pred_best_rf))
print("✅ Optimized Random Forest Accuracy:", accuracy_score(y_test, y_pred_best_rf))


✅ Best Parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
📊 Optimized Random Forest Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       0.75      1.00      0.86         3
           2       1.00      1.00      1.00         6
           3       1.00      0.80      0.89         5

    accuracy                           0.97        30
   macro avg       0.94      0.95      0.94        30
weighted avg       0.97      0.97      0.97        30

✅ Optimized Random Forest Accuracy: 0.9666666666666667


In [27]:
from xgboost import XGBClassifier

# Train XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', scale_pos_weight=10, random_state=42)
xgb_model.fit(X_train_balanced, y_train_balanced)

# Make predictions
y_pred_xgb = xgb_model.predict(X_test)

# Evaluate performance
from sklearn.metrics import classification_report, accuracy_score
print("📊 XGBoost Classification Report:\n", classification_report(y_test, y_pred_xgb))
print("✅ XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))


📊 XGBoost Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.94      0.97        16
           1       0.75      1.00      0.86         3
           2       1.00      1.00      1.00         6
           3       0.80      0.80      0.80         5

    accuracy                           0.93        30
   macro avg       0.89      0.93      0.91        30
weighted avg       0.94      0.93      0.94        30

✅ XGBoost Accuracy: 0.9333333333333333


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:45:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [28]:
import joblib

# Train and save model
rf_model.fit(X_train_balanced, y_train_balanced)
joblib.dump(rf_model, "random_forest_risk_model.pkl")

xgb_model.fit(X_train_balanced, y_train_balanced)
joblib.dump(xgb_model, "xgboost_risk_model.pkl")

print("✅ Models saved successfully!")



✅ Models saved successfully!


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:45:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [29]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(xgb_model, X_train_balanced, y_train_balanced, cv=5, scoring="accuracy")

print("📊 Cross-Validation Accuracy Scores:", cv_scores)
print("✅ Average Accuracy:", cv_scores.mean())


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:45:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:45:13] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:45:13] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:45:13] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


📊 Cross-Validation Accuracy Scores: [0.96153846 0.98076923 0.98076923 1.         0.96153846]
✅ Average Accuracy: 0.9769230769230768


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:45:13] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [30]:
from sklearn.model_selection import train_test_split

# Define feature columns (X) and target columns (y)
X = df_recipes.drop(columns=["Flammability (Yes/No)", "Reactivity (Stable/Unstable)",
                             "Explosiveness (1-10)", "Health Risk Score (0-100)",
                             "Risk Score (0-100)", "Risk Category Encoded"])

y = df_recipes[["Flammability (Yes/No)", "Reactivity (Stable/Unstable)", "Explosiveness (1-10)",
                "Health Risk Score (0-100)", "Risk Score (0-100)", "Risk Category Encoded"]]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("✅ Data split successfully!")


✅ Data split successfully!


In [31]:
# Vectorize Combined Recipe
X_train_text = vectorizer.fit_transform(X_train["Combined Recipe"]).toarray()
X_test_text = vectorizer.transform(X_test["Combined Recipe"]).toarray()

# Convert to DataFrames
X_train_text_df = pd.DataFrame(X_train_text, columns=vectorizer.get_feature_names_out(), index=X_train.index)
X_test_text_df = pd.DataFrame(X_test_text, columns=vectorizer.get_feature_names_out(), index=X_test.index)

# Drop original text column
X_train.drop(columns=["Combined Recipe"], inplace=True)
X_test.drop(columns=["Combined Recipe"], inplace=True)

# Merge TF-IDF Features
X_train = pd.concat([X_train.reset_index(drop=True), X_train_text_df.reset_index(drop=True)], axis=1)
X_test = pd.concat([X_test.reset_index(drop=True), X_test_text_df.reset_index(drop=True)], axis=1)

print("✅ Combined Recipe converted into TF-IDF features!")


✅ Combined Recipe converted into TF-IDF features!


In [32]:
tfidf_features = vectorizer.transform(new_recipe["Combined Recipe"]).toarray()
print("🔍 TF-IDF Feature Vector:", tfidf_features)


NameError: name 'new_recipe' is not defined